<a href="https://colab.research.google.com/github/SandeeeeeeeeepDey/data-science-11-weeks-progg/blob/main/california_housing_sequential_torch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
from sklearn.datasets import fetch_california_housing
import pandas as pd

In [3]:
cali_house_data = fetch_california_housing()
cali_data = pd.DataFrame(cali_house_data['data'], columns = cali_house_data["feature_names"])
cali_target = pd.Series(cali_house_data['target'], name = cali_house_data["target_names"][0])

In [4]:
train_size = round(0.8*len(cali_data))
test_size = round(0.1*len(cali_data))
valid_size = round(0.1*len(cali_data))

In [5]:
X_train, y_train = cali_data[:train_size], cali_target[:train_size]
X_valid, y_valid = cali_data[train_size : (train_size + valid_size)], cali_target[train_size : (train_size + valid_size)]
X_test, y_test = cali_data[(train_size + valid_size):], cali_target[(train_size + valid_size):]

In [6]:
means_train = X_train.values.mean(axis = 0)
means_valid = X_valid.values.mean(axis = 0)
means_test = X_test.values.mean(axis = 0)

In [7]:
stds_train = X_train.values.std(axis = 0)
stds_valid = X_valid.values.std(axis = 0)
stds_test = X_test.values.std(axis = 0)

In [8]:
import torch

class CaliforniaHousing(Dataset):
  def __init__(self, data, target, means, stds):
    self.data = torch.tensor(data, dtype=torch.float32)
    self.target = torch.tensor(target).float()
    self.means = torch.tensor(means).float()
    self.stds = torch.tensor(stds).float()
    self.data = (self.data - self.means) / self.stds
    # self.data = (data.float - self.means) / self.stds

  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):

    sample = {'data': self.data[idx], 'target': self.target[idx]}


    return sample

In [9]:
ch_train = CaliforniaHousing(X_train.values, y_train.values, means_train, stds_train) # .values for turning df to np arrays, that the model understands
ch_valid = CaliforniaHousing(X_valid.values, y_valid.values, means_valid, stds_train)
ch_test = CaliforniaHousing(X_test.values, y_test.values, means_test, stds_test)

In [10]:
len(ch_train)

16512

In [11]:
ch_train[0]

{'data': tensor([ 2.4176,  0.9166,  0.5983, -0.1517, -0.9612, -0.0881,  1.2584, -1.6117]),
 'target': tensor(4.5260)}

In [12]:
train_dl = DataLoader(ch_train, shuffle = True, batch_size = 32)
valid_dl = DataLoader(ch_train, shuffle = True, batch_size = 32)
test_dl = DataLoader(ch_test, shuffle = True, batch_size = 32)

In [13]:
batch = next(iter(train_dl))
batch

{'data': tensor([[-7.9647e-01,  7.6024e-01, -6.0680e-01, -1.9506e-01,  9.4637e-01,
           1.7645e-01, -5.6756e-01,  5.6750e-01],
         [-7.0441e-01, -4.1265e-01, -2.3526e-01,  1.3274e-02, -8.0699e-01,
          -2.5514e-01,  1.1865e+00, -1.4505e+00],
         [-6.9928e-01, -1.7807e-01,  1.2589e+01,  1.5919e+01, -1.1551e+00,
          -1.5651e-01,  1.7664e+00, -4.1548e-01],
         [ 8.1560e-01, -8.0361e-01,  1.4109e+00,  2.1942e-01, -1.1516e+00,
           1.1674e-01, -5.9152e-01,  8.3275e-01],
         [ 1.9382e-01, -1.5855e+00,  2.3605e-01, -5.0626e-02, -1.0478e-01,
           1.0693e-01, -5.3880e-01,  9.0556e-01],
         [-1.0677e+00, -9.5999e-01, -1.7104e-01,  5.0543e-02, -1.9870e-01,
           2.6364e-02,  7.5038e-01, -2.9066e-01],
         [-6.0134e-01, -3.3445e-01,  3.4977e-01,  3.5558e-01, -6.4501e-01,
          -1.0316e-01,  1.7568e+00, -4.5709e-01],
         [ 2.5051e+00,  8.3843e-01,  5.5860e-01, -2.3834e-01, -2.4695e-01,
          -6.5741e-03, -5.3401e-01,  5.571

In [32]:
batch['data'].shape, batch['target'][0]

(torch.Size([32, 8]), tensor(7.8312))

In [15]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

In [16]:
class CalNet(nn.Module):
  def __init__(self):
    super().__init__()
    self.dense_relu_stack = nn.Sequential(
        nn.Linear(8,50),
        nn.ReLU(),
        nn.Linear(50,50),
        nn.ReLU(),
        nn.Linear(50,50),
        nn.ReLU(),
        nn.Linear(50, 1)
    )

  def forward(self, x):
    logits = self.dense_relu_stack(x)
    return logits

cali_model = CalNet().to(device)
cali_model

CalNet(
  (dense_relu_stack): Sequential(
    (0): Linear(in_features=8, out_features=50, bias=True)
    (1): ReLU()
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): ReLU()
    (4): Linear(in_features=50, out_features=50, bias=True)
    (5): ReLU()
    (6): Linear(in_features=50, out_features=1, bias=True)
  )
)

best practice

In [17]:
# Sanity check:
output = cali_model(batch['data'])
output.shape

torch.Size([32, 1])

In [18]:
loss_fn = nn.MSELoss()
# loss_fn = nn.
optimizer = torch.optim.Adam(cali_model.parameters(),lr = 1e-3)

In [23]:
def train(model, dataloader, loss_fn, optimizer):
  size = len(dataloader.dataset)
  model.train()

  for batch_idx, batch in enumerate(dataloader):
    X,y = batch['data'].to(device), batch['target'].to(device)
    pred = model(X)
    loss = loss_fn(pred.squeeze(1), y) #squeeze(1) to make [32,1] to [32]

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    if batch_idx % 100 == 0:
      loss, current = loss.item(), (batch_idx+1)*len(X)

      print(f"loss:{loss:>8f}, current:{current:>5d}, size:{size:>5d}")

In [27]:
def validate(model, dataloader, loss_fn):
  size = len(dataloader.dataset)
  num_batch = len(dataloader)

  model.eval()

  loss_val, correct = 0,0
  with torch.no_grad():
    for batch in dataloader:
      X,y = batch["data"].to(device), batch["target"].to(device)
      pred = model(X)

      loss_val += loss_fn(pred.squeeze(1), y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()
  loss_val /=  num_batch
  correct /= size

  print(f"acc:{100*correct:>0.1f}, loss:{loss_val:>8f}")

In [28]:
def test(model, dataloader, loss_fn):
  size = len(dataloader.dataset)
  num_batch = len(dataloader)

  model.eval()
  loss_test, correct = 0,0

  with torch.no_grad():
    for batch in dataloader:
      X, y = batch["data"].to(device), batch["target"].to(device)

      pred = model(X)
      print(pred," diff-->", y)
      loss_test += loss_fn(pred.squeeze(1), y).item()
      correct += (pred.argmax(1) == y).type(torch.float). sum().item()
  loss_test /= num_batch
  correct /= size

  print(f"acc:{100*correct:>0.1f}")

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([16])) that is different to the input size (torch.Size([16, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)

In [29]:
epoch = 200

for i in range(epoch):
  print(f"Epoch:{i+1}")
  train(cali_model, train_dl, loss_fn, optimizer)
  validate(cali_model, valid_dl, loss_fn)

test(cali_model, test_dl, loss_fn)

Epoch:1
loss:0.292042, current:   32, size:16512
loss:0.482216, current: 3232, size:16512
loss:0.246681, current: 6432, size:16512
loss:0.155132, current: 9632, size:16512
loss:0.276047, current:12832, size:16512
loss:0.152669, current:16032, size:16512
acc:0.0, loss:0.306754
Epoch:2
loss:0.287048, current:   32, size:16512
loss:0.331336, current: 3232, size:16512
loss:0.211630, current: 6432, size:16512
loss:0.172121, current: 9632, size:16512
loss:0.321348, current:12832, size:16512
loss:0.214743, current:16032, size:16512
acc:0.0, loss:0.285646
Epoch:3
loss:0.108882, current:   32, size:16512
loss:0.251402, current: 3232, size:16512
loss:0.311947, current: 6432, size:16512
loss:0.305246, current: 9632, size:16512
loss:0.189495, current:12832, size:16512
loss:0.462756, current:16032, size:16512
acc:0.0, loss:0.279941
Epoch:4
loss:0.365196, current:   32, size:16512
loss:0.225367, current: 3232, size:16512
loss:0.150405, current: 6432, size:16512
loss:0.225194, current: 9632, size:165